<a href="https://colab.research.google.com/github/DrMelissaFranklin/DrMelissaFranklin/blob/main/Mel_HousingB_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Housing Harbinger
Project 2: Housing Price Prediction
The project should be done using Regression prediction techniques ( with feature scaling and regularization ) as covered in class. The goal is to minimize the RMS percentage error ( root mean squared percentage error - RMSPE ) on your prediction of the house sales price.

Any of the available features can be used in the regression, but a minimum set of variables that do the job should be the ultimate goal.

Be sure to go through the whole data science process and document as such in your Jupyter notebook.

This project will have less direct "To Do" guidance and the progression through the data science process will be more individualized this time around. We will talk about all the issues during class so you're not going to be out on a ledge with this one, I just want to move you towards performing a data science project on your own eventually.

A data dictionary file is available at AWS S3 at Housing Data Dictionary.

The data is available on AWS S3 at https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Data/Housing.Data.csv .

Use the grading rubric and the Project Blank. Refer to Project 1 -- but no accuracy or confusion matrices...

Drop the unneeded columns as you see fit. [Start with small number, add it more? like the numpy 8 lec?]. given a data dictionary: do not trust it! verify it with value counts / nunique (no unknowns, 9999, strings of words)

pandas, head info describe etc

## Problem Definition


State the business problem. Translate the business problem into a Data Science problem by stating what kind of problem it is ( supervised vs unsupervised ) and whether it is a classification, regression, or clustering problem.

The goal is to take passenger information based on ticket records for the Titanic and predict whether or not a passenger would have survived.  This is a binary classification problem since each passenger either survived or didn't survive.  This project will use Naive Bayes classification.  

## Data Collection/Sources


In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

In [ ]:
url = "https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Data/Housing.Data.csv"
house_orig = pd.read_csv( url )
house_orig.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,905101070,20,RL,62.0,14299,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,7,2007,WD,Normal,115400
1,905101330,90,RL,72.0,10791,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,Shed,500,10,2006,WD,Normal,90000
2,903454090,50,RM,50.0,9000,Pave,NaN,Reg,Bnk,AllPub,...,0,NaN,NaN,NaN,0,12,2007,WD,Normal,141000
3,533244030,60,FV,68.0,7379,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,254000
4,909252020,70,RL,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,4,2009,WD,Normal,155000


## Data Cleaning


In [ ]:
house_orig.shape

(2637, 81)

In [ ]:
house_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PID              2637 non-null   int64  
 1   MS SubClass      2637 non-null   int64  
 2   MS Zoning        2637 non-null   object 
 3   Lot Frontage     2188 non-null   float64
 4   Lot Area         2637 non-null   int64  
 5   Street           2637 non-null   object 
 6   Alley            180 non-null    object 
 7   Lot Shape        2637 non-null   object 
 8   Land Contour     2637 non-null   object 
 9   Utilities        2637 non-null   object 
 10  Lot Config       2637 non-null   object 
 11  Land Slope       2637 non-null   object 
 12  Neighborhood     2637 non-null   object 
 13  Condition 1      2637 non-null   object 
 14  Condition 2      2637 non-null   object 
 15  Bldg Type        2637 non-null   object 
 16  House Style      2637 non-null   object 
 17  Overall Qual  

In [ ]:
categorical_cols = house_orig.select_dtypes(include=['object']).columns
num_categorical_cols = len(categorical_cols)
print(f"Number of categorical columns: {num_categorical_cols}")

Number of categorical columns: 43


In [ ]:
for col in categorical_cols:
    num_categories = house_orig[col].nunique()
    print(f"Column '{col}' has {num_categories} categories.")

Column 'MS Zoning' has 7 categories.
Column 'Street' has 2 categories.
Column 'Alley' has 2 categories.
Column 'Lot Shape' has 4 categories.
Column 'Land Contour' has 4 categories.
Column 'Utilities' has 3 categories.
Column 'Lot Config' has 5 categories.
Column 'Land Slope' has 3 categories.
Column 'Neighborhood' has 28 categories.
Column 'Condition 1' has 9 categories.
Column 'Condition 2' has 8 categories.
Column 'Bldg Type' has 5 categories.
Column 'House Style' has 8 categories.
Column 'Roof Style' has 6 categories.
Column 'Roof Matl' has 8 categories.
Column 'Exterior 1st' has 16 categories.
Column 'Exterior 2nd' has 17 categories.
Column 'Mas Vnr Type' has 4 categories.
Column 'Exter Qual' has 4 categories.
Column 'Exter Cond' has 5 categories.
Column 'Foundation' has 6 categories.
Column 'Bsmt Qual' has 5 categories.
Column 'Bsmt Cond' has 5 categories.
Column 'Bsmt Exposure' has 4 categories.
Column 'BsmtFin Type 1' has 6 categories.
Column 'BsmtFin Type 2' has 6 categories.
C

In [ ]:
# Get categorical columns
categorical_cols = house_orig.select_dtypes(include=['object']).columns

# Loop through each categorical column
for col in categorical_cols:
    print(f"\nValue counts for column '{col}':")

    # Get value counts for each category within the column
    value_counts_series = house_orig[col].value_counts()

    # Print the value counts
    print(value_counts_series)


Value counts for column 'MS Zoning':
MS Zoning
RL         2043
RM          419
FV          123
C (all)      25
RH           23
I (all)       2
A (agr)       2
Name: count, dtype: int64

Value counts for column 'Street':
Street
Pave    2625
Grvl      12
Name: count, dtype: int64

Value counts for column 'Alley':
Alley
Grvl    111
Pave     69
Name: count, dtype: int64

Value counts for column 'Lot Shape':
Lot Shape
Reg    1662
IR1     895
IR2      65
IR3      15
Name: count, dtype: int64

Value counts for column 'Land Contour':
Land Contour
Lvl    2365
HLS     111
Bnk     106
Low      55
Name: count, dtype: int64

Value counts for column 'Utilities':
Utilities
AllPub    2634
NoSewr       2
NoSeWa       1
Name: count, dtype: int64

Value counts for column 'Lot Config':
Lot Config
Inside     1924
Corner      459
CulDSac     160
FR2          81
FR3          13
Name: count, dtype: int64

Value counts for column 'Land Slope':
Land Slope
Gtl    2511
Mod     112
Sev      14
Name: count, dtype:

In [ ]:
house_orig.isnull().sum()

,0
PID,0
MS SubClass,0
MS Zoning,0
Lot Frontage,449
Lot Area,0
...,...
Mo Sold,0
Yr Sold,0
Sale Type,0
Sale Condition,0


In [ ]:
# Find columns with nulls
cols_with_nulls = house_orig.columns[house_orig.isnull().any()].tolist()

# Calculate and print percentage of nulls for each column
for col in cols_with_nulls:
    null_percentage = (house_orig[col].isnull().sum() / len(house_orig)) * 100
    print(f"Column '{col}': {null_percentage:.2f}% nulls")

Column 'Lot Frontage': 17.03% nulls
Column 'Alley': 93.17% nulls
Column 'Mas Vnr Type': 60.94% nulls
Column 'Mas Vnr Area': 0.87% nulls
Column 'Bsmt Qual': 2.77% nulls
Column 'Bsmt Cond': 2.77% nulls
Column 'Bsmt Exposure': 2.88% nulls
Column 'BsmtFin Type 1': 2.77% nulls
Column 'BsmtFin SF 1': 0.04% nulls
Column 'BsmtFin Type 2': 2.81% nulls
Column 'BsmtFin SF 2': 0.04% nulls
Column 'Bsmt Unf SF': 0.04% nulls
Column 'Total Bsmt SF': 0.04% nulls
Column 'Bsmt Full Bath': 0.08% nulls
Column 'Bsmt Half Bath': 0.08% nulls
Column 'Fireplace Qu': 48.69% nulls
Column 'Garage Type': 5.50% nulls
Column 'Garage Yr Blt': 5.57% nulls
Column 'Garage Finish': 5.57% nulls
Column 'Garage Cars': 0.04% nulls
Column 'Garage Area': 0.04% nulls
Column 'Garage Qual': 5.57% nulls
Column 'Garage Cond': 5.57% nulls
Column 'Pool QC': 99.58% nulls
Column 'Fence': 79.98% nulls
Column 'Misc Feature': 96.36% nulls


In [ ]:
house_orig.columns

Index(['PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area', 'Street',
       'Alley', 'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config',
       'Land Slope', 'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Year Built',
       'Year Remod/Add', 'Roof Style', 'Roof Matl', 'Exterior 1st',
       'Exterior 2nd', 'Mas Vnr Type', 'Mas Vnr Area', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2',
       'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air',
       'Electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Fireplace Qu',
       'Garage Type', 'Garage Yr Blt', 'Garage Finish'

In [ ]:
house_orig.describe()

,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2.637000e+03,2637.000000,2188.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2614.000000,2636.000000,...,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000
mean,7.141301e+08,57.349261,69.166819,10044.694729,6.097459,5.569966,1971.288586,1984.202882,101.887911,438.441199,...,94.305271,46.984452,22.813424,2.368601,15.775123,2.130072,42.014031,6.243838,2007.795601,179986.230186
std,1.887527e+08,42.499091,23.356779,6742.549521,1.411522,1.118262,30.306986,20.913077,179.578232,449.602326,...,126.927272,66.564333,61.177638,23.160600,55.783751,35.140140,393.158781,2.722093,1.306403,78309.251522
min,5.263011e+08,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,5.284770e+08,20.000000,58.000000,7436.000000,5.000000,5.000000,1954.000000,1965.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129500.000000
50%,5.354530e+08,50.000000,68.000000,9450.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,368.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,160000.000000
75%,9.071870e+08,70.000000,80.000000,11526.000000,7.000000,6.000000,2001.000000,2004.000000,164.000000,732.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,213000.000000
max,1.007100e+09,190.000000,313.000000,164660.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1424.000000,742.000000,584.000000,407.000000,576.000000,800.000000,12500.000000,12.000000,2010.000000,745000.000000


In [ ]:
# Define the columns
A = ['Alley']
B = ['Street', 'Utilities', 'Has_Alley']

# Count values for A and B
# Instead of using A + B directly as index, use it to select columns from categorical_cols
counts_A = categorical_cols[categorical_cols.isin(A + B)].value_counts(dropna=False).reset_index()
counts_B = categorical_cols[categorical_cols.isin(B)].value_counts(dropna=False).reset_index()

# Rename the 'index' column to 'feature' in both dataframes
counts_A = counts_A.rename(columns={'index': 'feature'})
counts_B = counts_B.rename(columns={'index': 'feature'})

# Merge the dataframes on the 'feature' column which now contains the values from A and B
merged_df = pd.merge(
    counts_A,
    counts_B,
    on='feature', # Merge on the common 'feature' column
    suffixes=('_A', '_B'),
    how='outer'  # Consider using 'outer' to keep all rows from both dataframes
)

# Calculate marginal probabilities where count_B is not 0
merged_df['marg_prob'] = merged_df.apply(lambda row: (row['count_A'] / row['count_B']) * 100 if row['count_B'] != 0 else 0, axis=1)

# Display the resulting dataframe
merged_df

,feature,count_A,count_B,marg_prob
0,Alley,1,NaN,NaN
1,Street,1,1.0,100.0
2,Utilities,1,1.0,100.0


In [ ]:
#code from Joe to look at what's correlated:

corrs = housing_numeric.corr()['SalePrice'].abs().sort_values(ascending = False)
keep = corrs[(corrs>.3) & (corrs <1)]
keep

In [ ]:
house_orig.tail()

## Exploratory Data Analysis


## Processing


## Data Visualization/Communication of Results
